## Patching 3D

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pylops

from pylops.signalprocessing.patch3d import patch3d_design
from pylops.utils.describe import describe

from pylops.signalprocessing import Patch3D
from patch3dol import Patch3D as Patch3DOLD

In [14]:
nwin = (42, 42, 34)
nover = (10, 10, 4)
nop = (64, 64, 64)
dimsd = (500, 100, 200) # small
#dimsd = (1000, 500, 500) # large
tapertype = 'cosine'

y = np.arange(dimsd[0]*dimsd[1]*dimsd[2]).reshape(dimsd)

nwins, dims, mwin_inends, dwin_inends = patch3d_design(dimsd, nwin, nover, nop)

# no operator broadcast
Op = pylops.signalprocessing.FFTND(nwin, nffts=nop, workers=4)
Slid = Patch3DOLD(Op.H, dims, dimsd, nwin, nover, nop, tapertype='cosine')
Slid1a = Patch3D(Op.H, dims, dimsd, nwin, nover, nop, tapertype='cosine')

# with operator broadcast
Op = pylops.signalprocessing.FFTND((*nwins, *nwin), nffts=nop, workers=4)
Slid1b = Patch3D(Op.H, dims, dimsd, nwin, nover, nop, tapertype='cosine')

x = Slid.H * y.ravel()

In [15]:
print(np.allclose(Slid @ x, Slid1a @ x), np.allclose(Slid.H @ y, Slid1a.H @ y))
print(np.allclose(Slid @ x, Slid1b @ x), np.allclose(Slid.H @ y, Slid1b.H @ y))

True True
True True


In [16]:
%timeit -n 2 -r 3 Slid * x # OLD
%timeit -n 2 -r 3 Slid1a * x # NEW
%timeit -n 2 -r 3 Slid1b * x # NEW with Op broadcasted

1.84 s ± 298 ms per loop (mean ± std. dev. of 3 runs, 2 loops each)
462 ms ± 24 ms per loop (mean ± std. dev. of 3 runs, 2 loops each)
846 ms ± 79.4 ms per loop (mean ± std. dev. of 3 runs, 2 loops each)


In [17]:
%timeit -n 2 -r 3 Slid.H * y # OLD
%timeit -n 2 -r 3 Slid1a.H * y # NEW
%timeit -n 2 -r 3 Slid1b.H * y # NEW with Op broadcasted

860 ms ± 77.9 ms per loop (mean ± std. dev. of 3 runs, 2 loops each)
583 ms ± 11.3 ms per loop (mean ± std. dev. of 3 runs, 2 loops each)
432 ms ± 9.85 ms per loop (mean ± std. dev. of 3 runs, 2 loops each)
